In [13]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression


In [12]:
mass_balance_df = pd.read_csv('project-glaciers/data/mass_balance_hy.csv')
davos_dev = pd.read_csv('project-glaciers/data/weather_dev_davos_temp.csv')
davos_dev = davos_dev.drop(columns = 'hydrological year')
silvretta_mb = mass_balance_df[mass_balance_df['glacier name'] == 'Silvrettagletscher']
silvretta_mb_hy = silvretta_mb[['annual mass balance (mm w.e.)']]
silvretta_mb_w = silvretta_mb[['winter mass balance (mm w.e.)']]
silvretta_mb_s = silvretta_mb[['summer mass balance (mm w.e.)']]
silvretta_mb_hy

,annual mass balance (mm w.e.)
0,-415
1,381
2,-28
3,-414
4,1022
...,...
106,-888
107,-3339
108,-2309
109,-1654


In [15]:
# Merge on index
silvretta_reg_t = silvretta_mb_hy.merge(davos_dev, left_index=True, right_index=True)

# Define predictors and target
X = silvretta_reg_t[['october_td', 'november_td', 'december_td', 
                         'january_td', 'february_td', 'march_td',
                         'april_td', 'may_td', 'june_td', 'july_td',
                         'august_td', 'september_td'
                    ]] 
Y = silvretta_reg_t['annual mass balance (mm w.e.)']

# Add intercept for statsmodels
X = sm.add_constant(X)

# Fit regression
model = sm.OLS(Y, X).fit()

print(model.summary())

                                  OLS Regression Results                                 
Dep. Variable:     annual mass balance (mm w.e.)   R-squared:                       0.633
Model:                                       OLS   Adj. R-squared:                  0.588
Method:                            Least Squares   F-statistic:                     14.11
Date:                           Thu, 20 Nov 2025   Prob (F-statistic):           1.51e-16
Time:                                   17:29:34   Log-Likelihood:                -858.10
No. Observations:                            111   AIC:                             1742.
Df Residuals:                                 98   BIC:                             1777.
Df Model:                                     12                                         
Covariance Type:                       nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------

In [16]:
silvretta_reg_t_s = silvretta_mb_s.merge(davos_dev, left_index=True, right_index=True)

# Define predictors and target
X = silvretta_reg_t[['may_td', 'june_td', 'july_td',
                         'august_td', 'september_td'
                    ]] 
Y = silvretta_reg_t['summer mass balance (mm w.e.)']

# Add intercept for statsmodels
X = sm.add_constant(X)

# Fit regression
model = sm.OLS(Y, X).fit()

print(model.summary())

KeyError: 'summer mass balance (mm w.e.)'